# Stochastic Nonparametric Envelopment of Data (StoNED)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 16, 2020

### Objective: Estimates a log-transformed cost function model and technicial inefficiency

In [18]:
# import package pystoned
from pystoned import *

import pandas as pd
import numpy as np

### Prepare and read data

Download the Excel data from <https://github.com/ds2010/StoNED-Python/tree/master/Data>

Data description is now available at <https://github.com/ds2010/StoNED-Python/blob/master/Data/data_description.ipynb>

In [19]:
# import data
df = pd.read_excel ('data.xlsx')

In [20]:
# output (total cost)
y  = df['TOTEX']
y  = y.to_numpy()
y  = y.tolist()

Note 1: Converting Numpy array to list by `array.tolist()` can speed up the computation.

In [21]:
# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)
x   = x.tolist()

### Stage 1: Convex Nonparametric Least Squares estimation

In [22]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

crt   = "mult"
func  = "cost"
pps   = "crs"

model = CNLS.cnls(y, x, crt, func, pps)
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

Note 2:
     # crt     = "addi" : Additive composite error term
     #         = "mult" : Multiplicative composite error term
     # func    = "prod" : production frontier
     #         = "cost" : cost frontier
     # pps     = "vrs"  : variable returns to scale production possibility sets (pps)
     #         = "crs"  : constant returns to scale pps

In [23]:
# display estimates
model.e.display() 
model.b.display() 

e : residuals
    Size=89, Index=i
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
      0 :  None :    0.03591704716740125 :  None : False : False :  Reals
      1 :  None :   0.026035484499695317 :  None : False : False :  Reals
      2 :  None :     0.2021966194208569 :  None : False : False :  Reals
      3 :  None :  -0.004517966599345962 :  None : False : False :  Reals
      4 :  None :    0.17430052203661542 :  None : False : False :  Reals
      5 :  None :   -0.11912248827552518 :  None : False : False :  Reals
      6 :  None :   -0.01130133790753906 :  None : False : False :  Reals
      7 :  None :    0.11921720099488699 :  None : False : False :  Reals
      8 :  None :     0.3736984392624628 :  None : False : False :  Reals
      9 :  None :  -0.040184225000433275 :  None : False : False :  Reals
     10 :  None :    0.02303955229234552 :  None : False : False :  Reals
     11 :  None :   -0.07779539297949795 :  None : False : False :  Reals
   

In [24]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

### Stage 2: Residuals decomposition

In [25]:
# decomposition by method od moment (MoM)
method      = 'MoM'
res         = StoNED.stoned(y, eps, func, method, crt)

Note 3:

     # func    = "prod": production frontier;  
     #         = "cost": cost frontier
     # method  = "MOM" : Method of moments
     #         = "QLE" : Quasi-likelihood estimation 
     #         = "NKD" : Nonparametric kernel deconvolution (TBA...)
     # crt     = "addi": Additive composite error term
     #         = "mult": Multiplicative composite error term

In [30]:
# display technical inefficiency
TE = res[1]
TE

array([0.89538104, 0.89828469, 0.83594277, 0.90676489, 0.8470783 ,
       0.93218984, 0.90854604, 0.86787864, 0.76397426, 0.91572402,
       0.89914948, 0.92411954, 0.81710042, 0.81218697, 0.91206415,
       0.93661185, 0.83886568, 0.84904684, 0.90920393, 0.92834973,
       0.93359455, 0.95237175, 0.84204541, 0.92742919, 0.86704358,
       0.88640399, 0.81885786, 0.9436953 , 0.87284109, 0.89779573,
       0.90597635, 0.9665862 , 0.91688081, 0.83033677, 0.90663263,
       0.85265999, 0.94706703, 0.93793438, 0.93144484, 0.9149757 ,
       0.91761361, 0.89260188, 0.88637793, 0.92068577, 0.9264701 ,
       0.93427318, 0.89791042, 0.88013446, 0.93538945, 0.9345783 ,
       0.83841977, 0.91146366, 0.83666673, 0.83687434, 0.89329044,
       0.94732826, 0.94879926, 0.92634485, 0.9436191 , 0.89995288,
       0.94683983, 0.92465838, 0.91466649, 0.90128702, 0.76723302,
       0.9304141 , 0.88106876, 0.9177529 , 0.88075511, 0.94995516,
       0.9177128 , 0.88108738, 0.94397599, 0.92788296, 0.92616